# Opt algo visualizer

This notebook reads the pickle produced by `tests.py` and visualizes it

In [60]:
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [61]:
out = pickle.load(open('tests.pickle', 'rb'))
out.keys()
out[('DataModel', 'RandomSearch')][2]

,z,func,history,x
0,196,1.122214,"[[1, 3.622233695019733], [2, 3.251522790468373...",1.4792000989038963|1.9949072753879173|2.008765...
1,196,1.040437,"[[1, 2.0142006524724705], [2, 1.95014020570314...",-0.9031258108944016|-2.1155828326681814|-1.524...
2,196,1.079673,"[[1, 2.5754597419102363], [2, 2.17472243894895...",-0.408315582751253|-2.8305125642579156|-2.9958...
3,196,1.060885,"[[1, 3.2881578702251764], [2, 2.88961181697645...",1.667596838393905|1.7847125989388006|1.0510141...
4,196,1.066746,"[[1, 3.519733934816732], [2, 3.157050123517003...",0.7201696584218258|-0.34919402937200683|1.1308...


In [62]:
benchmark_to_plot = 'DataModel'
algo_to_plot      = 'RandomSearch'

The code below will plot the trace for this algorithm and this benchmark. Click on the pan button, and then
  - Left-click and drag to pan
  - Right-click and drag to zoom

In [63]:
import plotly.graph_objects as go

df = out[(benchmark_to_plot, algo_to_plot)][2].copy()
best_obj = out[(benchmark_to_plot, algo_to_plot)][1]

df_exploded = df.explode('history').reset_index(drop=True)
df_exploded['obj'] = df_exploded.history.str[1].astype(float)

# Create figure
fig = go.Figure()

# Add cumulative minimum line
fig.add_trace(
    go.Scatter(
        x=df_exploded.index,
        y=df_exploded.obj.cummin(),
        mode='lines+markers',
        name='Minimum thus far',
        line=dict(color='red', width=2),
        marker=dict(size=6, symbol='circle-open')
    )
)

# Add individual traces for each x value
for this_group, this_df in df_exploded.groupby('x'):
    fig.add_trace(
        go.Scatter(
            x=this_df.index,
            y=this_df.obj,
            mode='lines+markers',
            name=f'x={this_group[:20]}...',
            line=dict(color='black', width=0.3),
            marker=dict(size=4, symbol='x'),
            showlegend=False
        )
    )

# Update layout
fig.update_layout(
    title=f'Connected lines correspond to evaluations<br>for a single value of x at different fidelities<br>(Minimum thus far in red; final minimum: {round(best_obj, 3)})',
    xaxis_title='Function evaluation number',
    yaxis_title='Function value (log scale)',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True,
    width=1000,
    height=600
)

fig.show()

In [64]:
import numpy as np
x,y,history,_ = out[('DataModel', 'SMAC')]
x

print(x)
proportions = np.exp(x) / np.sum(np.exp(x))
print(proportions)
print(y)

[ 2.54145737 -1.12720378 -1.99975593  1.0274298  -2.79041218]
[0.79300372 0.02023    0.00845379 0.17447829 0.00383419]
0.8948295073667719


In [65]:
import numpy as np
x,y,history,_ = out[('DataModel', 'RandomSearch')]
x

print(x)
proportions = np.exp(x) / np.sum(np.exp(x))
print(proportions)
print(y)

[-0.90312581 -2.11558283 -1.52496898 -2.21114293  2.01779289]
[0.04839544 0.01439597 0.02598612 0.01308398 0.89813849]
1.040436509419981


In [66]:
import plotly.graph_objects as go
import numpy as np

# Specify which benchmark to analyze
benchmark_to_plot = 'DataModel'

# Create figure
fig = go.Figure()

colors = {
    'SMAC': 'orange',
    'RandomSearch': 'black',
    'GridSearch': 'green',
}

# Plot each optimizer's performance
for algo in ['SMAC', 'RandomSearch', 'GridSearch']:
    # Get the data for this optimizer
    df = out[(benchmark_to_plot, algo)][2].copy()
    
    # Explode the history column to get all evaluations
    df_exploded = df.explode('history').reset_index(drop=True)
    df_exploded['obj'] = df_exploded.history.str[1].astype(float)
    
    # Calculate cumulative budget (time) - each evaluation costs its fidelity value
    df_exploded['budget'] = df_exploded.history.str[0].astype(float)
    df_exploded['cumulative_time'] = df_exploded['budget'].cumsum()
    
    # Add trace for this optimizer
    fig.add_trace(
        go.Scatter(
            x=df_exploded['cumulative_time'],  # wall clock time
            y=df_exploded.obj.cummin(),  # running minimum
            mode='lines',
            name=algo,
            line=dict(color=colors[algo], width=2),
        )
    )

# Update layout
fig.update_layout(
    title=f'Optimizer Comparison on {benchmark_to_plot}',
    xaxis_title='Cumulative Wall Clock Time',
    yaxis_title='Best Function Value Found (log scale)',
    yaxis_type='log',
    xaxis_type='log',  # log scale for time axis
    template='plotly_white',
    showlegend=True,
    width=1000,
    height=600,
    xaxis=dict(
        gridwidth=1,
        gridcolor='LightGrey',
        showgrid=True,
    ),
    yaxis=dict(
        gridwidth=1,
        gridcolor='LightGrey',
        showgrid=True,
    )
)

fig.show()

In [67]:
import numpy as np
import pandas as pd

benchmark_to_plot = 'DataModel'

# Analysis for each optimizer
for algo in ['SMAC', 'RandomSearch', 'GridSearch']:
    print(f"\n=== Analysis for {algo} ===")
    
    # Get the data
    df = out[(benchmark_to_plot, algo)][2].copy()
    df_exploded = df.explode('history').reset_index(drop=True)
    df_exploded['fidelity'] = df_exploded.history.str[0].astype(float)
    df_exploded['obj'] = df_exploded.history.str[1].astype(float)
    
    # Basic stats
    print(f"Total number of evaluations: {len(df_exploded)}")
    print(f"Total compute cost (sum of fidelities): {df_exploded['fidelity'].sum()}")
    
    # Fidelity distribution
    print("\nFidelity distribution:")
    print(f"Min fidelity: {df_exploded['fidelity'].min()}")
    print(f"Max fidelity: {df_exploded['fidelity'].max()}")
    print(f"Mean fidelity: {df_exploded['fidelity'].mean():.2f}")
    print(f"Median fidelity: {df_exploded['fidelity'].median()}")
    
    # Count evaluations at different fidelity ranges
    fidelity_ranges = [0, 50, 100, 150, 200]
    for i in range(len(fidelity_ranges)-1):
        count = len(df_exploded[(df_exploded['fidelity'] > fidelity_ranges[i]) & 
                               (df_exploded['fidelity'] <= fidelity_ranges[i+1])])
        print(f"Evaluations with fidelity {fidelity_ranges[i]}-{fidelity_ranges[i+1]}: {count}")
    
    # Final performance
    print(f"\nBest objective value found: {df_exploded['obj'].min():.4f}")


=== Analysis for SMAC ===
Total number of evaluations: 2266
Total compute cost (sum of fidelities): 87900.0

Fidelity distribution:
Min fidelity: 1.0
Max fidelity: 196.0
Mean fidelity: 38.79
Median fidelity: 18.0
Evaluations with fidelity 0-50: 1668
Evaluations with fidelity 50-100: 310
Evaluations with fidelity 100-150: 150
Evaluations with fidelity 150-200: 138

Best objective value found: 0.8948

=== Analysis for RandomSearch ===
Total number of evaluations: 980
Total compute cost (sum of fidelities): 96530.0

Fidelity distribution:
Min fidelity: 1.0
Max fidelity: 196.0
Mean fidelity: 98.50
Median fidelity: 98.5
Evaluations with fidelity 0-50: 250
Evaluations with fidelity 50-100: 250
Evaluations with fidelity 100-150: 250
Evaluations with fidelity 150-200: 230

Best objective value found: 1.0404

=== Analysis for GridSearch ===
Total number of evaluations: 980
Total compute cost (sum of fidelities): 96530.0

Fidelity distribution:
Min fidelity: 1.0
Max fidelity: 196.0
Mean fidelit

In [68]:
import numpy as np
import pandas as pd

benchmark_to_plot = 'DataModel'

def calculate_true_compute(df):
    """Calculate true compute cost accounting for memoization"""
    compute_cost = 0
    seen_configs = {}  # Keep track of configurations we've seen
    
    # Convert x column to string for consistent dictionary keys
    df['x_str'] = df['x'].astype(str)
    
    # Go through each evaluation in order
    for _, row in df.iterrows():
        x_str = row['x_str']
        history = row['history']  # Already a list of [fidelity, value] pairs
        
        if x_str not in seen_configs:
            # New configuration - count all fidelities
            compute_cost += sum(float(h[0]) for h in history)
            seen_configs[x_str] = float(history[-1][0])  # highest fidelity used
        else:
            # Configuration seen before - only count new fidelities
            prev_max_fidelity = seen_configs[x_str]
            new_fidelities = [float(h[0]) for h in history if float(h[0]) > prev_max_fidelity]
            compute_cost += sum(new_fidelities)
            seen_configs[x_str] = max(float(history[-1][0]), prev_max_fidelity)
            
    return compute_cost

# Analysis for each optimizer
for algo in ['SMAC', 'RandomSearch', 'GridSearch']:
    print(f"\n=== Analysis for {algo} ===")
    
    # Get the data
    df = out[(benchmark_to_plot, algo)][2].copy()
    
    # Calculate true compute cost with memoization
    true_compute = calculate_true_compute(df)
    
    # Get evaluation statistics
    df_exploded = df.explode('history').reset_index(drop=True)
    df_exploded['fidelity'] = df_exploded.history.str[0].astype(float)
    df_exploded['obj'] = df_exploded.history.str[1].astype(float)
    
    print(f"Number of unique configurations tested: {len(df['x'].unique())}")
    print(f"Total number of evaluations (including repeated configs): {len(df_exploded)}")
    print(f"True compute cost (accounting for memoization): {true_compute}")
    
    # Fidelity distribution of actual evaluations
    print("\nFidelity distribution of actual evaluations:")
    print(f"Min fidelity: {df_exploded['fidelity'].min()}")
    print(f"Max fidelity: {df_exploded['fidelity'].max()}")
    print(f"Mean fidelity: {df_exploded['fidelity'].mean():.2f}")
    print(f"Median fidelity: {df_exploded['fidelity'].median()}")
    
    # Best performance
    print(f"\nBest objective value found: {df_exploded['obj'].min():.4f}")


=== Analysis for SMAC ===
Number of unique configurations tested: 132
Total number of evaluations (including repeated configs): 2266
True compute cost (accounting for memoization): 87900.0

Fidelity distribution of actual evaluations:
Min fidelity: 1.0
Max fidelity: 196.0
Mean fidelity: 38.79
Median fidelity: 18.0

Best objective value found: 0.8948

=== Analysis for RandomSearch ===
Number of unique configurations tested: 5
Total number of evaluations (including repeated configs): 980
True compute cost (accounting for memoization): 96530.0

Fidelity distribution of actual evaluations:
Min fidelity: 1.0
Max fidelity: 196.0
Mean fidelity: 98.50
Median fidelity: 98.5

Best objective value found: 1.0404

=== Analysis for GridSearch ===
Number of unique configurations tested: 5
Total number of evaluations (including repeated configs): 980
True compute cost (accounting for memoization): 96530.0

Fidelity distribution of actual evaluations:
Min fidelity: 1.0
Max fidelity: 196.0
Mean fidelit